In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models

# Reading file name from the test annotation

In [8]:
y_s=pd.read_excel('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/annotation_test_01.xlsx')
fname_s=y_s.fname

# Preprocessing the test samples

In [12]:
max=0
for i in range(len(fname_s)):
  #give path where test data is stored
  path="/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/test/" + fname_s[i]
  X_test_s=np.load(path)
  if X_test_s.shape[2]>max:
    max=X_test_s.shape[2]

In [15]:
# Reshaping test data
for i in range(len(fname_s)):
  #give path where test data is stored
  path='/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/test/'+fname_s[i]
  X_test_s=np.load(path)
  X_test_s=np.reshape(X_test_s,(X_test_s.shape[1],X_test_s.shape[2]))
  X_test_s=np.append(X_test_s,np.zeros([X_test_s.shape[0],(max-X_test_s.shape[1])]),axis=1)
  #Store the corrected test samples
  path1='/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/testcorrected/'+fname_s[i]
  np.save(path1,X_test_s)

In [16]:
# Stacking test data
X_s=np.zeros([len(fname_s),X_test_s.shape[0],max])
# y_s=np.zeros(len(fname_s))
for i in range(len(fname_s)):
  path='/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/testcorrected/'+ fname_s[i]
  d_s=np.load(path)
  # d_s=np.reshape(d_s,(128,2584))
  X_s[i,:]=d_s

# Saving files and then loading of test


In [18]:
np.save('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/cnn_data_test/X_s.npy',X_s)
# np.save('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/X_test.npy',X_test)
# np.save('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/y_train.npy',y_train)
# np.save('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/cnn_data_test/y_s.npy',y)


In [19]:
X_s=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/sir/cnn_data_test/X_s.npy')
# X_test=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/X_test.npy')
# y_train=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/y_train.npy')
# y=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data_s/y.npy')

# load files of training

In [3]:
X=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/X.npy')
# X_test=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/X_test.npy')
# y_train=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/y_train.npy')
y=np.load('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/cnn_data/y.npy')

# For training model use orignal 1000 files

In [4]:
cnn = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(5, 5), activation='relu', input_shape=(X.shape[1],X.shape[2],1)),
    layers.MaxPooling2D((3, 3)),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # layers.Conv2D(filters=8, kernel_size=(2, 2), activation='relu'),
    # layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [5]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
c=cnn.fit( X,y,batch_size=32,epochs=10)

# Now predict test data using this model

In [ ]:
y_pred_s=cnn.predict(X_s)

In [ ]:
#Converting into int by finding max
y_pred_classes_s=[np.argmax(element) for element in y_pred_s]

In [ ]:
#Converting predicted labels as int into strings
list=[]
arr=['Walk_and_footsteps','Bark', 'Crying_and_sobbing', 'Doorbell', 'Knock', 'Meow','Microwave_oven', 'Shatter', 'Siren','Vehicle_horn_and_car_horn_and_honking']
for i in range(len(y_pred_classes_s)):
  list.append(arr[y_pred_classes_s[i]])

# Storing predicted labels onto csv file

In [ ]:
y_s.split=list
y_s.to_csv('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/annotations_3.csv')

In [ ]:
# y_pred=cnn.predict(X_test)
# y_pred_classes=[np.argmax(element) for element in y_pred]


# Making classification report

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print('classification report\n',classification_report(y_test,y_pred_classes))

# Finding confusion matrix

In [ ]:
cm=np.zeros([10,10],dtype='uint8')
for i in range(10):
  for j in range(10):
    t=0
    for k in range(len(y_pred_classes)):      
      if ((j==y_pred_classes[k]) and (i==y_test[k])):
        t=t+1
    cm[i,j]=t

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred_classes)
cm

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
import matplotlib.pyplot as plt
acc=c.history['accuracy']
loss=c.history['loss']
val_loss=c.history['val_loss']
for i in range(len(val_loss)):
  val_loss[i]=val_loss[i]/2
val_acc=c.history['val_accuracy']
plt.figure(1)
plt.title('Accuracy vs Epochs')
plt.plot(acc,"g-")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/acc.jpg')
plt.figure(2)
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(loss,'r-')
plt.savefig('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/loss.jpg')
plt.figure(3)
plt.title('Validation accuracy and loss')
plt.xlabel('Epochs')
plt.plot(val_loss,'g-')
plt.plot(val_acc,'b-')
plt.legend(['val_loss','val_acc'])
plt.savefig('/content/drive/MyDrive/SPML/Audio_Classification-MLSP/val.jpg')